**Author:** Matt Sarmiento

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploration" data-toc-modified-id="Exploration-1">Exploration</a></span></li></ul></div>

In [1]:
# Third-party imports
import pandas as pd

# Exploration

Let's look for trends in the county data for overdoses from 1999-2017. Specifically, we want to know
* Which counties had the highest age-adjusted-death-rate in 2017?
* Which counties have experienced the largest increase in adge-adjusted-death-rates? Are they urban or rural?


In [4]:
# Load data
county = pd.read_csv('../input/deaths-by-county-1999-2017.csv')
county.head(3)

,FIPS,Year,State,FIPS State,County,Population,Model-based Death Rate,Standard Deviation,Lower Confidence Limit,Upper Confidence Limit,Urban/Rural Category,Census Division
0,1001,2003,Alabama,1,"Autauga County, AL","46,800",6.752270,1.532372,4.394907,10.370797,Medium Metro,6
1,1001,2004,Alabama,1,"Autauga County, AL","48,366",6.948645,1.449189,4.644302,10.302841,Medium Metro,6
2,1001,2005,Alabama,1,"Autauga County, AL","49,676",6.587989,1.376595,4.399895,9.774890,Medium Metro,6


In [8]:
county.sort_values(by="Model-based Death Rate", ascending=False).head(10)

,FIPS,Year,State,FIPS State,County,Population,Model-based Death Rate,Standard Deviation,Lower Confidence Limit,Upper Confidence Limit,Urban/Rural Category,Census Division
44881,54011,2017,West Virginia,54,"Cabell County, WV","94,958",130.247391,10.794249,110.232284,152.549195,Medium Metro,5
31409,39113,2017,Ohio,39,"Montgomery County, OH","531,542",95.238119,4.142941,87.330880,103.574214,Medium Metro,3
45541,54099,2017,West Virginia,54,"Wayne County, WV","40,153",92.792251,12.142881,71.279715,118.835221,Medium Metro,5
45149,54047,2015,West Virginia,54,"McDowell County, WV","19,704",92.152497,14.803818,66.854439,124.788647,Noncore,5
18164,24510,2017,Maryland,24,"Baltimore city, MD","611,648",87.566894,3.707845,80.484553,95.022599,Large Central Metro,5
44880,54011,2016,West Virginia,54,"Cabell County, WV","95,669",87.160807,8.500652,71.590822,104.911036,Medium Metro,5
45610,54109,2011,West Virginia,54,"Wyoming County, WV","23,455",84.776075,13.292553,61.945712,113.972597,Noncore,5
45136,54045,2017,West Virginia,54,"Logan County, WV","32,925",84.467393,12.102523,63.325811,110.707905,Noncore,5
45614,54109,2015,West Virginia,54,"Wyoming County, WV","22,173",81.208486,13.045920,58.918360,109.972850,Noncore,5
18163,24510,2016,Maryland,24,"Baltimore city, MD","616,958",79.920225,3.521102,73.202304,87.007753,Large Central Metro,5
